In [1]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
import random
from statsmodels.stats.proportion import binom_test
from statsmodels.stats.multitest import multipletests
import os

In [2]:
def btest(pa1, pa2, seed=0, return_proportions=False):
    """ Performs genome wide binomial test between two groups of taxa
    Parameters
    ----------
    df1 : pd.DataFrame
        Rows are taxa, columns are genes
    df2 : pd.DataFrame
        Rows are taxa, columns are genes
    Returns
    -------
    pd.Series : list of genes associated with df1
    pd.Series : list of genes associated with df2
    """
    np.random.seed(seed)
    random.seed(seed)
    #pa1 = df1 > 0
    #pa2 = df2 > 0
    idx = list(set(pa1.columns) | set(pa2.columns))
    idx.sort()
    pa1 = pa1.sum(axis=0).reindex(idx).fillna(0)
    pa2 = pa2.sum(axis=0).reindex(idx).fillna(0)
    n = pa1 + pa2
    obs = list(zip(list(pa1.values), list((pa2.values + 1) / (pa2 + 1).sum()), list(n.values)))
    pvals = pd.Series([binom_test(a, n, b, 'two-sided') for (a, b, n) in obs],
                      index=n.index)
    if return_proportions:
        res = pd.DataFrame({'groupA': pa1, 'groupB': pa2, 'pval': pvals})
        def relabel_f(x):
            if x['groupA'] < x['groupB']:
                return 'groupB'
            else:
                return 'groupA'
        res['side'] = res.apply(relabel_f, axis=1)
        return res

    return pvals

In [3]:
def parse_genome(df):
    genome_id = df['#query'][0].split('_')[0]
    keggs = df['KEGG_ko'].replace('-', None).dropna()
    keggs = list(map(lambda x: x.split(','), keggs.values))
    keggs = sum(keggs, [])
    keggs = pd.DataFrame({'KEGG_ko': keggs})
    keggs['genome_id'] = genome_id
    return keggs


In [4]:
def get_kegg_from_annotation(ids):
    """
    ids: Species_rep of the top/bottom k microbes
    """
    df_list = []
    for i in ids:
        f_name = "../../permutation_table/Species/{}_eggNOG.tsv".format(i)
        try:
            df_parsed = parse_genome(pd.read_table(f_name))
        except Exception as e:
            print("error in parsing ", f_name)
            raise e
        df_list.append(df_parsed)
    df_cat = pd.concat(df_list, axis = 0)
    kegg_counts = to_sparse_matrix(df_cat)
    return kegg_counts
 

In [13]:
G_matrix = get_kegg_from_annotation(ids_nodup)

In [17]:
df_G_matrix = pd.DataFrame(G_matrix) 
df_G_matrix.to_csv('../../permutation_table/G_matrix.txt', index = True, sep='\t' ) 
df_G_matrix

,ko:K09121,ko:K14063,ko:K06122,ko:K14155,ko:K02670,ko:K14089,ko:K11908,ko:K01641,ko:K12700,ko:K03572,...,ko:K00670,ko:K02637,ko:K02358,ko:K18700,ko:K02922,ko:K05798,ko:K01126,ko:K15654,ko:K01190,ko:K06948
MGYG000003131,1,0,0,2,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0
MGYG000000129,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,0,0
MGYG000003657,2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
MGYG000000846,2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
MGYG000002104,2,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGYG000003886,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
MGYG000004474,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,2,0,1,0
MGYG000002585,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,2,0,7,0
MGYG000003394,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [7]:
ids = M["Species_rep"]
ids_nodup = ids.drop_duplicates()
ids_nodup

0       MGYG000000001
1       MGYG000000002
2       MGYG000000003
3       MGYG000000004
4       MGYG000000005
            ...      
4861    MGYG000004891
4865    MGYG000004895
4868    MGYG000004898
4871    MGYG000004901
4874    MGYG000004904
Name: Species_rep, Length: 4744, dtype: object

In [8]:
ids_nodup = [_test_top_id for _test_top_id in ids_nodup if "." not in _test_top_id]
len(ids_nodup)

4728

In [54]:
df = pd.DataFrame(ids_nodup) 
df.to_csv('../../permutation_table/ids_nodup.csv') 
df

,0
0,MGYG000000001
1,MGYG000000002
2,MGYG000000003
3,MGYG000000004
4,MGYG000000005
...,...
4723,MGYG000004891
4724,MGYG000004895
4725,MGYG000004898
4726,MGYG000004901


In [58]:
df = pd.read_csv('../../permutation_table/ids_nodup.csv', index_col = 0)
df['0']

0       MGYG000000001
1       MGYG000000002
2       MGYG000000003
3       MGYG000000004
4       MGYG000000005
            ...      
4723    MGYG000004891
4724    MGYG000004895
4725    MGYG000004898
4726    MGYG000004901
4727    MGYG000004904
Name: 0, Length: 4728, dtype: object

In [41]:
ids_1185 = list(set(ids_nodup) - set(AD_micrbes_ids))

In [12]:
def to_sparse_matrix(func_df, genome_id='genome_id', kegg_id='KEGG_ko'):
    # create genome-specific index
    ogus = list(set(func_df[genome_id]))
    ogu_lookup = pd.Series(np.arange(0, len(ogus)), ogus)
    # create KEGG-specific index
    keggs = list(set(func_df[kegg_id]))
    kegg_lookup = pd.Series(np.arange(0, len(keggs)), keggs)
    # rename names as numbers
    ogu_id = func_df[genome_id].apply(lambda x: ogu_lookup.loc[x]).astype(np.int64)
    kegg_id = func_df[kegg_id].apply(lambda x: kegg_lookup.loc[x]).astype(np.int64)
    # assign the presence / absence of a gene
    func_df['count'] = 1
    c = func_df['count'].values
    # format into a matrix
    data = coo_matrix((c, (ogu_id, kegg_id)))
    ko_ogu = pd.DataFrame(data.todense(), index=ogus, columns=keggs)
    return ko_ogu

In [6]:
table_AD = pd.read_table("../../permutation_table/Laske2022AD_new.txt", index_col = 0) 

In [11]:
AD_microbes = set(table_AD.index)

In [20]:
AD_micrbes_ids = M[M['Species'].isin(AD_microbes)]
AD_micrbes_ids = AD_micrbes_ids.drop_duplicates()
AD_micrbes_ids = AD_micrbes_ids['Species_rep']
AD_micrbes_ids = [_test_top_id for _test_top_id in AD_micrbes_ids if "." not in _test_top_id]
AD_micrbes_ids #3556
len(AD_micrbes_ids) #3543

3543

In [12]:
table_AD = pd.read_table("../../permutation_table/Laske2022AD_new.txt", index_col = 0) 
table_ad_T = table_AD.T
table_ad_T

species,1XD42-69 sp014287635,28L sp000177555,43-108 sp001915545,51-20 sp001917175,51-20 sp900539605,51-20 sp900542055,51-20 sp900762565,AF33-28 sp003477885,AM07-15 sp003477405,AM51-8 sp003478275,...,Zag1 sp900752845,Zag1 sp900767135,Zag1 sp900769355,Zag1 sp900769645,Zag111 sp002102825,Zag111 sp002103105,Zag111 sp003258735,Zag111 sp004558955,Zag111 sp900551965,Zag111 sp900555325
ERR9578158,4848,0,233,50,13,83,197,1771,2170,4716,...,97,75,722,99,43,35,0,16,109,53
ERR9578316,22084,0,242,2448,44,463,274,1865,1096,561,...,219,218,130,172,0,0,23,22,23,135
ERR9578225,1664,11,969,0,0,0,210,1655,5429,1850,...,0,464,5651,43,0,0,83,0,0,0
ERR9578267,1379,0,533,17,0,22,0,1465,1436,538,...,181,419,5188,98,34,15,36,0,0,10
ERR9578253,446,0,470,0,0,0,0,1894,4302,633,...,0,0,360,21,0,0,242,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR9578173,1139,0,37,0,0,0,0,3744,4292,1222,...,0,158,1274,186,19,0,282,0,19,0
ERR9578300,908,0,262,64,22,298,11,2765,2222,18981,...,249,647,70,511,35,98,77,69,262,166
ERR9578239,537,0,650,270,364,56,59,4563,2500,379,...,25,0,233,210,77,28,39,88,136,71789
ERR9578296,364,0,171,0,0,0,0,514,1129,640,...,0,0,212,0,0,0,0,0,14,0


In [13]:
table_ad_T.to_csv("../../permutation_table/table_AD_microbes.txt", sep = "\t")


In [21]:
#select 10 microbes as a test table
table_test_ad = table_AD#.head(10)

In [17]:
#filter metadata
#Metadata = pd.read_table("../.. table/metadata_12dataset_1258_adonis.txt", index_col = 0)

In [53]:
Metadata_AD = Metadata[(Metadata['Study'] == "Laske2022AD")]

In [22]:
#Metadata_AD
table_test_ad_T = table_test_ad.T

In [27]:
table_test_ad_T.reindex(Metadata_AD.index)
table_test_ad_T

species,1XD42-69 sp014287635,28L sp000177555,43-108 sp001915545,51-20 sp001917175,51-20 sp900539605,51-20 sp900542055,51-20 sp900762565,AF33-28 sp003477885,AM07-15 sp003477405,AM51-8 sp003478275,...,Zag1 sp900752845,Zag1 sp900767135,Zag1 sp900769355,Zag1 sp900769645,Zag111 sp002102825,Zag111 sp002103105,Zag111 sp003258735,Zag111 sp004558955,Zag111 sp900551965,Zag111 sp900555325
ERR9578158,4848,0,233,50,13,83,197,1771,2170,4716,...,97,75,722,99,43,35,0,16,109,53
ERR9578316,22084,0,242,2448,44,463,274,1865,1096,561,...,219,218,130,172,0,0,23,22,23,135
ERR9578225,1664,11,969,0,0,0,210,1655,5429,1850,...,0,464,5651,43,0,0,83,0,0,0
ERR9578267,1379,0,533,17,0,22,0,1465,1436,538,...,181,419,5188,98,34,15,36,0,0,10
ERR9578253,446,0,470,0,0,0,0,1894,4302,633,...,0,0,360,21,0,0,242,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR9578173,1139,0,37,0,0,0,0,3744,4292,1222,...,0,158,1274,186,19,0,282,0,19,0
ERR9578300,908,0,262,64,22,298,11,2765,2222,18981,...,249,647,70,511,35,98,77,69,262,166
ERR9578239,537,0,650,270,364,56,59,4563,2500,379,...,25,0,233,210,77,28,39,88,136,71789
ERR9578296,364,0,171,0,0,0,0,514,1129,640,...,0,0,212,0,0,0,0,0,14,0


In [34]:
table_test_ad_T.to_csv("../../permutation_table/table_AD_test.txt", sep = "\t")
Metadata_AD.to_csv("../../permutation_table/metadata_AD_test.txt", sep = "\t")

In [6]:
M  = pd.read_table("/Users/dongminjin/Documents/github_rep/permutation_table/eggNOG_species_rep.txt", sep = '\t')

In [37]:
Metadata_AD = pd.read_csv("../../permutation_table/metadata_AD_test.txt", sep = "\t", index_col = 1)
Metadata_AD

,Unnamed: 0,disease,Study,gender,age,Batch
featureid,,,,,,
ERR9578157,0,AD,Laske2022AD,Female,58.0,1
ERR9578158,1,AD,Laske2022AD,Female,78.0,1
ERR9578159,2,AD,Laske2022AD,Female,73.0,1
ERR9578160,3,AD,Laske2022AD,Male,77.0,1
ERR9578161,4,AD,Laske2022AD,Female,73.0,1
...,...,...,...,...,...,...
ERR9578327,145,AD,Laske2022AD,Female,68.0,1
ERR9578328,146,AD,Laske2022AD,Male,67.0,1
ERR9578329,147,AD,Laske2022AD,Male,61.0,1


In [137]:
a = set(Metadata_AD["disease"])
a.remove('Healthy')
a
set(Metadata_AD["disease"]).remove('Healthy')

{'AD'}

In [144]:
', '.join(a)

'AD'

In [150]:
def _test(X, y, k, p, M, condition):
    """
    X: count matrix of microbes (m microbes X n samples)
    y: disease lables for samples (n samples)
    G: gene table
    k: top k microbes, eg k = 100
    p: number of permutations, eg p = 1000
    M: metadata = pd.read_table('../table/eggNOG_species_rep.txt')
    condition: colname in y states the disease status: disease, Healthy
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
        X, 
        y, 
        design_factors = condition,# compare samples based on the "disease"
        refit_cooks = True,
        n_cpus = 8
    )
    dds.deseq2()
    #set of the sample status: Healthy, disease
    status = set(y[condition])
    #remove healthy
    status.remove("Healthy")
    disease = ', '.join(status)
    #sample status colname: disease, 
    #disease: name of that disease
    stat_res = DeseqStats(dds,contrast = [condition, disease, "Healthy" ], n_cpus=8)
    #stat_res = DeseqStats(dds,contrast = ["disease", "Healthy", "AD" ], n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    #top k microbes: the ones with largest CI_5
    top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
    #bottom k microbes: the ones with smallest CI_95
    bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
    #convert microbe names to species ids
    top_microbe = set(top.index)
    bot_microbe = set(bot.index)
    top_rep = M[M['Species'].isin(top_microbe)]
    bot_rep = M[M['Species'].isin(bot_microbe)]
    top_id = top_rep['Species_rep'].drop_duplicates()
    top_id = [_test_top_id for _test_top_id in top_id if "." not in _test_top_id]
    bot_id = bot_rep['Species_rep'].drop_duplicates()
    bot_id = [_test_bot_id for _test_bot_id in bot_id if "." not in _test_bot_id]
    #MGYG000003543.1
    #wget all ids: dir: /mnt/home/djin/ceph/Permutation_test/table/Species
    #get G gene matrix
    #print(top_id)
    #print(bot_id)
    #print(type(bot_id))
    top_gene_matrix = get_kegg_from_annotation(top_id)
    bot_gene_matrix = get_kegg_from_annotation(bot_id)

    #binomial test Return number of genes elevated in top
    """
    C = btest(G[top], G[bot])
    return C
    """
    kegg = btest(top_gene_matrix,bot_gene_matrix,return_proportions=True)
    T = len(kegg)
    kegg_top = kegg.loc[kegg['side'] == 'groupA']
    #GWAS typically uses a threshold of 0.05 divided by the number of tests,
    #we have len(kegg) tests, so we could use  p = 0.05/T?
    kegg_top_sig = kegg_top.loc[kegg_top['pval'] <= 0.05/T] 
    C = len(kegg_top_sig.index)
    T = len(kegg)
    print(0.05/T)
    #print(kegg)
    return C, T

In [151]:
T2 = _test(table_test_ad_T, Metadata_AD, k=100, p=2, M=M, condition = "disease")
T2

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.39 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.41 seconds.

Fitting LFCs...
... done in 0.31 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.25 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896


7.396449704142013e-06


(1748, 6760)

In [115]:
T = _test(table_test_ad_T, Metadata_AD, k=100, p=2, M=M)

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.40 seconds.

Fitting dispersion trend curve...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.42 seconds.

Fitting LFCs...
... done in 0.39 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.22 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.24 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896


In [116]:
T

1748

In [112]:
    dds = DeseqDataSet(
        table_test_ad_T, 
        Metadata_AD, 
        design_factors = "disease",# compare samples based on the "disease"
        #reference_level = "AD",
        refit_cooks = True,
        n_cpus = 8
    )
    dds.deseq2()
    stat_res = DeseqStats(dds,contrast = ["disease", "Healthy", "AD"], n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    res_df

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.51 seconds.

Fitting dispersion trend curve...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.43 seconds.

Fitting LFCs...
... done in 0.32 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.22 seconds.

Fitting MAP dispersions...
... done in 0.23 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.27 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,-0.570896,0.257603,-2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,-0.668411,1.372424,-0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,-0.486055,0.244547,-1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,3.479790,0.637017,5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,1.575257,0.887438,1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608664,-2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,-0.730225,0.438464,-1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,-1.101342,0.803632,-1.370455,1.705449e-01,0.465896


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,CI_95,CI_5
species,,,,,,,,
1XD42-69 sp014287635,2130.637601,-0.570896,0.257603,-2.216188,2.667864e-02,0.158735,-0.065995,-1.075797
28L sp000177555,2.246988,-0.668411,1.372424,-0.487030,6.262371e-01,NaN,2.021539,-3.358362
43-108 sp001915545,500.585113,-0.486055,0.244547,-1.987571,4.685920e-02,0.231489,-0.006742,-0.965367
51-20 sp001917175,9710.835303,3.479790,0.637017,5.462633,4.691242e-08,0.000006,4.728343,2.231237
51-20 sp900539605,97.994360,1.575257,0.887438,1.775061,7.588781e-02,0.299433,3.314636,-0.164122
...,...,...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608664,-2.499349,1.244218e-02,0.098247,-0.328282,-2.714243
Zag111 sp003258735,51.116899,-0.730225,0.438464,-1.665416,9.582989e-02,0.340741,0.129165,-1.589615
Zag111 sp004558955,17.609086,-1.101342,0.803632,-1.370455,1.705449e-01,0.465896,0.473777,-2.676462


In [113]:
    dds = DeseqDataSet(
        table_test_ad_T, 
        Metadata_AD, 
        design_factors = "disease",# compare samples based on the "disease"
        #reference_level = "Healthy",
        refit_cooks = True,
        n_cpus = 8
    )
    dds.deseq2()
    stat_res = DeseqStats(dds,contrast = ["disease", "AD", "Healthy" ],n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    res_df

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.40 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.45 seconds.

Fitting LFCs...
... done in 0.35 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.24 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.27 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,CI_95,CI_5
species,,,,,,,,
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735,1.075797,0.065995
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN,3.358362,-2.021539
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489,0.965367,0.006742
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691242e-08,0.000006,-2.231237,-4.728343
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588781e-02,0.299433,0.164122,-3.314636
...,...,...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247,2.714243,0.328282
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741,1.589615,-0.129165
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896,2.676462,-0.473777


In [100]:
T = _test(table_test_ad_T, Metadata_AD, k=100, p=2, M=M)

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.41 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.42 seconds.

Fitting LFCs...
... done in 0.34 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.23 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.20 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216189,2.667854e-02,0.158417
28L sp000177555,2.246988,-0.668411,1.372523,-0.486995,6.262621e-01,NaN
43-108 sp001915545,500.585113,-0.486055,0.244549,-1.987556,4.686086e-02,0.231105
51-20 sp001917175,9710.835303,3.479790,0.637016,5.462638,4.691106e-08,0.000006
51-20 sp900539605,97.994360,1.575257,0.887428,1.775081,7.588449e-02,0.299038
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608696,-2.499216,1.244685e-02,0.098023
Zag111 sp003258735,51.116899,-0.730225,0.438479,-1.665357,9.584153e-02,0.340375
Zag111 sp004558955,17.609086,1.101343,0.803632,1.370457,1.705444e-01,0.465468


In [101]:
#reference_level = "Healthy",
T

4821

In [103]:
#reference_level = "AD",
T2 = _test(table_test_ad_T, Metadata_AD, k=100, p=2, M=M)
T2

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.39 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.44 seconds.

Fitting LFCs...
... done in 0.33 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.23 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.28 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,-0.570896,0.257603,-2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,-0.668411,1.372424,-0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,-0.486055,0.244547,-1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,3.479790,0.637017,5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,1.575257,0.887438,1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608664,-2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,-0.730225,0.438464,-1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,-1.101342,0.803632,-1.370455,1.705449e-01,0.465896


/Users/dongminjin/miniconda3/lib/python3.9/site-packages/scipy/stats/_discrete_distns.py:78: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


4640

In [33]:
Metadata_AD

,featureid,disease,Study,gender,age,Batch
0,ERR9578157,AD,Laske2022AD,Female,58.0,1
1,ERR9578158,AD,Laske2022AD,Female,78.0,1
2,ERR9578159,AD,Laske2022AD,Female,73.0,1
3,ERR9578160,AD,Laske2022AD,Male,77.0,1
4,ERR9578161,AD,Laske2022AD,Female,73.0,1
...,...,...,...,...,...,...
145,ERR9578327,AD,Laske2022AD,Female,68.0,1
146,ERR9578328,AD,Laske2022AD,Male,67.0,1
147,ERR9578329,AD,Laske2022AD,Male,61.0,1
148,ERR9578330,AD,Laske2022AD,Male,77.0,1


In [72]:
T = _test(table_test_ad_T, Metadata_AD, k=100, p=2, M=M)
#log2(Healthy/AD)

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.48 seconds.

Fitting dispersion trend curve...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.40 seconds.

Fitting LFCs...
... done in 0.37 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.16 seconds.

Running Wald tests...
... done in 0.29 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,-0.570896,0.257603,-2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,-0.668411,1.372424,-0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,-0.486055,0.244547,-1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,3.479790,0.637017,5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,1.575257,0.887438,1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608664,-2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,-0.730225,0.438464,-1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,-1.101342,0.803632,-1.370455,1.705449e-01,0.465896


/Users/dongminjin/miniconda3/lib/python3.9/site-packages/scipy/stats/_discrete_distns.py:78: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


In [67]:
T

(5145, 6760)

In [42]:
def permutation_test(X, y, k, p, M):
    T = _test(X, y, k, p, M)
    for i in range(p):
        #shuffle the group lables
        y_permutated = np.random.permutation(y['disease'])
        y_permutated = pd.DataFrame(y_permutated, index=y.index)
        y_permutated.columns = ['disease']
        y_permutated.reindex(X.index)
        T_ = _test(X, y_permutated, k, p, M)
    return T, (T > T_) / (p)
    #return T, (T > T_) / (p+1)

In [59]:
a, b = permutation_test(table_test_ad_T,Metadata_AD, k=100, p=2, M= M )

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.42 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.43 seconds.

Fitting LFCs...
... done in 0.34 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.25 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.26 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,-0.570896,0.257603,-2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,-0.668411,1.372424,-0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,-0.486055,0.244547,-1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,3.479790,0.637017,5.462633,4.691242e-08,0.000006
51-20 sp900539605,97.994360,1.575257,0.887438,1.775061,7.588781e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,-1.521263,0.608664,-2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,-0.730225,0.438464,-1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,-1.101342,0.803632,-1.370455,1.705449e-01,0.465896


/Users/dongminjin/miniconda3/lib/python3.9/site-packages/scipy/stats/_discrete_distns.py:78: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.40 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.44 seconds.

Fitting LFCs...
... done in 0.38 seconds.

Refitting 1782 outlier genes.

Fitting dispersions...
... done in 0.20 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.28 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.454357,0.258733,1.756086,0.079074,0.293505
28L sp000177555,1.865360,0.484039,1.403091,0.344980,0.730109,NaN
43-108 sp001915545,500.585113,-0.434847,0.245029,-1.774679,0.075951,0.284970
51-20 sp001917175,6170.306891,-2.080559,0.636886,-3.266766,0.001088,0.016371
51-20 sp900539605,83.393254,-1.189234,0.881618,-1.348922,0.177362,0.461850
...,...,...,...,...,...,...
Zag111 sp002103105,25.037912,-0.557536,0.563683,-0.989094,0.322617,0.635833
Zag111 sp003258735,42.072911,0.140430,0.422746,0.332186,0.739749,0.915675
Zag111 sp004558955,17.609086,-0.007517,0.812636,-0.009250,0.992620,NaN


Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.42 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.44 seconds.

Fitting LFCs...
... done in 0.37 seconds.

Refitting 1782 outlier genes.

Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.30 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,2130.637601,0.454357,0.258733,1.756086,0.079074,0.293505
28L sp000177555,1.865360,0.484039,1.403091,0.344980,0.730109,NaN
43-108 sp001915545,500.585113,-0.434847,0.245029,-1.774679,0.075951,0.284970
51-20 sp001917175,6170.306891,-2.080559,0.636886,-3.266766,0.001088,0.016371
51-20 sp900539605,83.393254,-1.189234,0.881618,-1.348922,0.177362,0.461850
...,...,...,...,...,...,...
Zag111 sp002103105,25.037912,-0.557536,0.563683,-0.989094,0.322617,0.635833
Zag111 sp003258735,42.072911,0.140430,0.422746,0.332186,0.739749,0.915675
Zag111 sp004558955,17.609086,-0.007517,0.812636,-0.009250,0.992620,NaN


In [60]:
a, b

(5145, 0.5)

In [192]:
y = Metadata_AD
counts = table_test_ad_T
y_permutated = np.random.permutation(y['disease'])
y_permutated = pd.DataFrame(y_permutated, index=y.index)
y_permutated.columns = ['disease']
y_permutated.reindex(counts.index)
y_permutated

,disease
featureid,
ERR9578157,Healthy
ERR9578158,AD
ERR9578159,Healthy
ERR9578160,AD
ERR9578161,Healthy
...,...
ERR9578327,AD
ERR9578328,AD
ERR9578329,Healthy


In [61]:
dds = DeseqDataSet(table_test_ad_T, Metadata_AD, design_factors = "disease", refit_cooks = True, n_cpus = 8)
dds.deseq2()
stat_res = DeseqStats(dds, n_cpus=8)
stat_res.summary()

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 4 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


In [62]:
M = pd.read_table('../table/eggNOG_species_rep.txt')

In [68]:
k = 100
top

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,CI_95,CI_5
species,,,,,,,,
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004,4.349303,1.918256
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191,0.453950,-0.166395
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191,0.430750,-0.275600
AM51-8 sp003478275,2345.324981,0.127499,0.235829,0.540641,5.887550e-01,0.797191,0.589724,-0.334726
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191,1.276786,-0.928645
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080,-0.138963,-0.975153
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884,0.048056,-1.124470
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191,2.333245,-1.163670
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191,0.768180,-1.548697


In [112]:
k = 2

In [113]:
res = stat_res.summary()
res_df = stat_res.results_df
res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
#top k microbes: the ones with largest CI_5
top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
#bottom k microbes: the ones with smallest CI_95
bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
#convert microbe names to species ids
top_microbe = set(top.index)
bot_microbe = set(bot.index)
top_rep = M[M['Species'].isin(top_microbe)]
bot_rep = M[M['Species'].isin(bot_microbe)]
top_id = top_rep['Species_rep'].drop_duplicates()
bot_id = bot_rep['Species_rep'].drop_duplicates()

Log2 fold change & Wald test p-value: disease Healthy vs AD


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
species,,,,,,
1XD42-69 sp014287635,1667.080704,-0.557058,0.213314,-2.611447,9.016003e-03,0.045080
28L sp000177555,1.770149,-0.364849,1.419733,-0.256985,7.971907e-01,0.797191
43-108 sp001915545,710.528771,-0.538207,0.299114,-1.799338,7.196526e-02,0.239884
51-20 sp001917175,6084.375509,3.133779,0.620165,5.053137,4.346110e-07,0.000004
51-20 sp900539605,72.803533,0.584788,0.892070,0.655540,5.121201e-01,0.797191
51-20 sp900542055,153.775373,0.174070,0.562610,0.309398,7.570187e-01,0.797191
51-20 sp900762565,184.591470,-0.390259,0.591040,-0.660291,5.090668e-01,0.797191
AF33-28 sp003477885,1773.295421,0.143777,0.158251,0.908537,3.635947e-01,0.797191
AM07-15 sp003477405,2885.886582,0.077575,0.180191,0.430513,6.668227e-01,0.797191


In [114]:
top_microbe

{'51-20 sp001917175', 'AF33-28 sp003477885'}

In [115]:
bot_microbe

{'1XD42-69 sp014287635', '43-108 sp001915545'}

In [116]:
#get G gene matrix
top_gene_matrix = get_kegg_from_annotation(top_id)
bot_gene_matrix = get_kegg_from_annotation(bot_id)


In [120]:
kegg = btest(top_gene_matrix,bot_gene_matrix,return_proportions=True)
kegg_top = kegg.loc[kegg['side'] == 'groupA']
kegg_top_sig = kegg_top.loc[kegg_top['pval'] <= 0.001]

In [126]:
len(kegg_top_sig.index)

1420

In [129]:
table_AD = pd.read_table("../table/Laske2022AD_new.txt", index_col = 0)
table_UC = pd.read_table("../table/Nielsen2014UC_new.txt", index_col = 0)
table_CD = pd.read_table("../table/Franzosa2019IBD_CD_new.txt", index_col = 0)

In [136]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}
set3 = {5, 6, 7}

combined_set = set1 | set2 | set3

print(combined_set)


{1, 2, 3, 4, 5, 6, 7}


In [138]:
set_AD = set(table_AD.index)
set_CD = set(table_CD.index)
set_UC = set(table_UC.index)

In [139]:
combined_microbes = set_AD | set_CD | set_UC

In [143]:
combined_microbes  = pd.DataFrame(combined_microbes) 

In [146]:
combined_microbes.to_csv('../table/microbes.csv') 

In [151]:
microbes = pd.read_csv('../table/microbes.csv', index_col = 0)
ids = set(microbes['0'])

In [153]:
ids_table = M[M['Species'].isin(ids)]

In [155]:
annotated_ids = ids_table['Species_rep'].drop_duplicates()

In [157]:
annotated_ids = pd.DataFrame(annotated_ids) 
annotated_ids.to_csv('../table/annotated_ids.csv') 

In [165]:
ids = pd.read_csv('../table/annotated_ids.csv', index_col = 0)
ids = set(ids['Species_rep'])

#/mnt/home/djin/ceph/Permutation_test/table/Species